In [118]:
import json
import math
from collections import defaultdict
import math

In [36]:
def load_data(data_path):
    with open(data_path, "r") as f:
        data = f.read()
    data = data.lower().replace("\n", " ")
    return data

In [37]:
data = load_data("data.txt")

In [121]:
# data

In [63]:
n_params = 10
state = [{} for i in range(n_params)]
vocab_size = len(set(data.split()))

In [87]:
train(data)

In [40]:
def train(data):
    training_data = data.split()
    for i in range(n_params):
        for idx, j in enumerate(training_data):
            if idx + i >= len(training_data):
                break
            context = tuple(training_data[idx: idx+i])
            next_token = training_data[idx+i]
            if context not in state[i]:
                state[i][context] = defaultdict(int)
            state[i][context][next_token] = state[i][context][next_token] + 1

In [119]:
def get_log_probs(state):
    log_probs = []
    for item in state:
        log_dict = {}
        for k, v in item.items():
            total_count = sum(v.values())
            log_dict[k] = {k1: math.log((v1+1) / (total_count + vocab_size)) for k1, v1 in v.items()}
        log_probs.append(log_dict)
    return log_probs

In [146]:
def get_perplexity(test_data, log_probs):
    test_tokens = test_data.lower().split()
    n_tokens = len(test_tokens)
    log_liklihood = 0
    if n_tokens > n_params:
        context = test_data[-n_params:]
    else:
        context = test_data
    for i in range(n_tokens):
        context = tuple(test_tokens[:i])
        token = test_tokens[i]
        log_liklihood += log_probs[i].get(context, {}).get(token, math.log(1e-10))
    avg_log_liklihood = log_liklihood / n_tokens
    perplexity = math.exp(-avg_log_liklihood)
    return perplexity

In [147]:
log_probs = get_log_probs(state)
data = get_perplexity("a quick brown", log_probs)